In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf

## Loading Data

In [15]:
data = pd.read_csv('all-data.csv',names=["Label","Text"],encoding='ISO-8859-1')
data

,Label,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   4846 non-null   object
 1   Text    4846 non-null   object
dtypes: object(2)
memory usage: 75.8+ KB


In [17]:
data.isnull().sum()

Label    0
Text     0
dtype: int64

In [18]:
data.describe()

,Label,Text
count,4846,4846
unique,3,4838
top,neutral,TELECOMWORLDWIRE-7 April 2006-TJ Group Plc sel...
freq,2879,2


## Preprocessing 

In [51]:
def get_sequences(texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    print(f"Vocab Length:{ len(tokenizer.word_index) + 1}")

    max_seq_length = np.max(list(map(lambda x:len(x),sequences)))
    print(f"Maximum sequence length:{max_seq_length}")
    sequences = pad_sequences(sequences,maxlen=max_seq_length,padding='post')
    return sequences

In [37]:
get_sequences(data['Text'])

Vocab Length:10123
Maximum sequence length:71


array([[  94,    5, 3498, ...,    0,    0,    0],
       [ 840,  336,    5, ...,    0,    0,    0],
       [   1,  293,  656, ...,    0,    0,    0],
       ...,
       [  42,   31,  242, ...,    0,    0,    0],
       [  30,   27,    2, ...,    0,    0,    0],
       [  27,    3,   35, ...,    0,    0,    0]])

In [38]:
get_sequences(data['Text']).shape

Vocab Length:10123
Maximum sequence length:71


(4846, 71)

In [39]:
def preprocess_input(df):
    df= df.copy()
    sequences = get_sequences(data['Text'])
    label_mapping = {
        'negative':0,
        'neutral':1,
        'positive':2
    }
    y = df['Label'].replace(label_mapping)
    
    train_sequences,test_sequences,y_train,y_test=train_test_split(sequences,y,train_size=0.7,shuffle=True,random_state=1)
    
    return train_sequences,test_sequences,y_train,y_test

In [40]:
train_sequences,test_sequences,y_train,y_test = preprocess_input(data)

Vocab Length:10123
Maximum sequence length:71


In [41]:
train_sequences

array([[5442,  510,   16, ...,    0,    0,    0],
       [  22, 1628,    4, ...,    0,    0,    0],
       [1141,  936,  136, ...,    0,    0,    0],
       ...,
       [   1,  419,   16, ...,    0,    0,    0],
       [2586,  123, 3247, ...,    0,    0,    0],
       [  30,  615,  555, ...,    0,    0,    0]])

In [43]:
y_train

545     2
2374    0
4217    1
1071    1
716     2
       ..
2895    1
2763    1
905     2
3980    1
235     2
Name: Label, Length: 3392, dtype: int64

## Training

In [46]:
inputs = tf.keras.Input(shape=(train_sequences.shape[1],))
x = tf.keras.layers.Embedding(
    input_dim=10123,
    output_dim=128,
    input_length=train_sequences.shape[1]
)(inputs)
x = tf.keras.layers.GRU(256, return_sequences=True, activation='tanh')(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_sequences,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
85/85 [==============================] - 30s 286ms/step - loss: 0.8313 - accuracy: 0.6377 - val_loss: 0.8123 - val_accuracy: 0.6348
Epoch 2/100
85/85 [==============================] - 13s 149ms/step - loss: 0.4777 - accuracy: 0.8116 - val_loss: 0.7907 - val_accuracy: 0.6701
Epoch 3/100
85/85 [==============================] - 8s 95ms/step - loss: 0.1665 - accuracy: 0.9399 - val_loss: 0.9976 - val_accuracy: 0.6701
Epoch 4/100
85/85 [==============================] - 8s 99ms/step - loss: 0.0484 - accuracy: 0.9864 - val_loss: 1.3249 - val_accuracy: 0.7099
Epoch 5/100
85/85 [==============================] - 9s 103ms/step - loss: 0.0190 - accuracy: 0.9956 - val_loss: 1.3737 - val_accuracy: 0.6834


In [47]:
results = model.evaluate(test_sequences, y_test, verbose=0)

print("Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

Test Loss: 0.69247
Test Accuracy: 72.15%
